### Introduction
In this demonstration, we show the basic steps needed to perform an elastostatic analysis of a concrete specimen using the PCG-MPI solver.

### Setup environment

To utilize the PCG-MPI solver, first install the required libraries and dependencies as outlined in the **`README.md file`** using the specified shell commands. In this project, the environment is set up by exporting the **`notebooks/setup_env.sh`** file as follows:

In [1]:
%%bash

cat <<EOL > setup_env.sh
source ~/env_MPI/bin/activate
export PYTHONPATH=~/env_MPI/lib/python3.11/site-packages:\$PYTHONPATH
module load python3/3.11.0
module load openmpi/4.0.5

export work_dir=~/PCG_MPI_Solver
cd \$work_dir

export n_meshparts=8

EOL


### Collect model data
We analyze an octree mesh model of a concrete specimen, depicted in Fig. 1. The mesh, generated from a 3D image of a concrete specimen measuring 512x512x512 voxels, was developed by Prof. Chongmin Song's research group at UNSW Sydney. In the mesh, blue cells represent aggregates, while red cells denote mortar. The model data—including the mesh, geometric details, constraints, and elastic material properties—is available in **`data/concrete.zip`**.

In [2]:
%%bash
source setup_env.sh

export model_name=concrete
export scratch_path=$work_dir/data
export input_model_path=$scratch_path/$model_name.zip

python3 src/data/read_input_model.py $work_dir $model_name $scratch_path $input_model_path


>creating directories..


>extracting files from concrete


>success!


>elements:  124693


>nodes:     208316 


>dofs:      624948


>total runtime: 2.06 sec


<div style="text-align: left;">
    <img src="images/mesh_material.png" alt="Mesh and Material" width="300"/>
    <p>Figure 1: An octree mesh generated from 3D image-based model of a concrete specimen</p>
</div>

### Partition mesh

The PCG-MPI solver leverages a mesh-partitioning strategy to enhance parallel processing efficiency. In this approach, the mesh model is divided into smaller sections, or mesh partitions. Within each partition, matrix operations are carried out locally on individual MPI processes, each utilizing a physical core. The data among the mesh-partitions is synchronised using inter-process communication.

In this demonstration, we configure the mesh partition count to 8 during the environment setup. For large-scale problems, this number can be substantially increased— e.g. 12,000 partitions for models with approximately 1 billion unknowns. Utilizing the Gadi supercomputer at NCI Canberra, which supports over 100,000 cores, enables us to perform massively parallel computations efficiently. To ensure high-quality mesh partitions, we employ the METIS package.

In [3]:
%%bash
source setup_env.sh

python3 src/solver/run_metis.py $n_meshparts



>loading model data..


>generating indices for 8 mesh parts..


>success!


>total runtime: 0.57 sec


<div style="text-align: left;">
    <img src="images/mesh_partition.png" alt="Mesh Partition" width="300"/>
    <p>Figure 2: Mesh partitions shown in different colours</p>
</div>

In [4]:
%%bash
source setup_env.sh

export n_cores=4   
mpiexec -np $n_cores --map-by numa python3 src/solver/partition_mesh.py $n_meshparts 0


>loading model data..


>partitioning mesh into 8 parts using 4 cores..


>success!


>total runtime: 3.05 sec


### Run PCG-MPI solver
We configure the time-history and solver parameters before running the PCG-MPI solver, as outlined below. This demonstration focuses on solving an elastostatic problem, and it outputs the nodal displacement of the model under specified constraints.

In [5]:
%%bash
source setup_env.sh

python3 -c "
from src.utils.file_operations import exportz

TimeHistoryParam={'ExportFlag': True, # export results
                  'ExportFrmRate' : 1,
                  'ExportFrms' : [],
                  'PlotFlag' : False,
                  'TimeStepDelta': [0,1],
                  'ExportVars': 'U'} #D U PS PE GS GE

SolverParam={'Tol' : 1e-7,
             'MaxIter' : 10000}

GlobSettings = {'TimeHistoryParam' : TimeHistoryParam,
                'SolverParam' : SolverParam}
exportz('__pycache__/GlobSettings.zpkl', GlobSettings)
"

In [6]:
%%bash
source setup_env.sh

mpiexec -np $n_meshparts --map-by numa python3 src/solver/pcg_solver.py 1 0


>loading partitioned data..


>running parallel pcg solver with 8 cores..


>exporting results..


>success!


>file read time:     0.2 sec 


>calculation time:   11.5 sec


>communication time: 1.0 sec


>---------------------------


>total runtime:      12.6 sec 


As we observe here, the communication time among MPI processes is notably low compared to the calculation time. The mesh-partitioning strategy effectively minimizes communication overhead, maintaining the parallel efficiency even as we scale up to thousands of cores for large-scale problems. Refer to the documents in **`docs/references.txt`** to gain more insights on parallel efficiencies (weak and strong).

### Export and visualize results
The displacement results produced by the solver are exported in a format conducive to model data visualization, such as .vtk. In this work, we use ParaView, which has a client–server architecture to facilitate remote visualization of large-scale datasets.

In [7]:
%%bash
source setup_env.sh

mpiexec -np 1 python3 src/data/export_vtk.py 1 "U" "Full"


>loading model and result data..


>exporting vtk files..


>success!


>total runtime: 2.27 sec


<img src="images/mesh_deformation.png" alt="Mesh Deformation" width="350"/>
Fig.3 Mesh deformation (scaled 1000 times for illustration)